# Independent Component Analysis

## Setup

In [1]:
import numpy as np

In [2]:
from scipy.io import wavfile

In [3]:
from scipy.signal import resample

In [7]:
import sys

In [14]:
from io import StringIO

In [15]:
import base64

In [16]:
import struct

In [18]:
from IPython.display import display

In [19]:
from IPython.core.display import HTML

In [22]:
%matplotlib inline
import pylab as pl

## Functions

In [26]:
def load_wav(filename, samplerate=1411000):
    rate, data = wavfile.read(filename)
    
    if len(data.shape) > 1:
        data = data[:,0]/2 + data[:,1]/2
    
    ratio = float(samplerate)/float(rate)
    data = resample(data, len(data)*ratio)
    
    return samplerate, data.astype(np.int16)

In [21]:
def wavPlayer(data, rate):
    
    buffer = StringIO.StringIO()
    buffer.write(b'RIFF')
    buffer.write(b'\x00\x00\x00\x00')
    buffer.write(b'WAVE')

    buffer.write(b'fmt ')
    if data.ndim == 1:
        noc = 1
    else:
        noc = data.shape[1]
    bits = data.dtype.itemsize * 8
    sbytes = rate*(bits // 8)*noc
    ba = noc * (bits // 8)
    buffer.write(struct.pack('<ihHIIHH', 16, 1, noc, rate, sbytes, ba, bits))

    buffer.write(b'data')
    buffer.write(struct.pack('<i', data.nbytes))

    if data.dtype.byteorder == '>' or (data.dtype.byteorder == '=' and sys.byteorder == 'big'):
        data = data.byteswap()

    buffer.write(data.tostring())

    size = buffer.tell()
    buffer.seek(4)
    buffer.write(struct.pack('<i', size-8))
    
    val = buffer.getvalue()
    
    src = """
    <head>
    <meta http-equiv="Content-Type" content="text/html; charset=utf-8">
    <title>Simple Test</title>
    </head>
    
    <body>
    <audio controls="controls" style="width:600px" >
      <source controls src="data:audio/wav;base64,{base64}" type="audio/wav" />
      Your browser does not support the audio element.
    </audio>
    </body>
    """.format(base64=base64.encodestring(val))
    display(HTML(src))

## Data Manipulation

In [ ]:
fs1,s1 = load_wav('Wavs/Hillary.wav')


pl.figure(figsize=(6.75,2))
pl.plot(s1)
pl.title('Signal 1')
pl.show()

wavPlayer(s1, fs1)